In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from functools import reduce
from scipy.stats import linregress
from scipy import fftpack
from scipy.signal import detrend
from tqdm import tqdm
from scipy.optimize import root_scalar
from tqdm.notebook import tqdm_notebook
from scipy.optimize import curve_fit
from tsmoothie.smoother import LowessSmoother
import matplotlib
from statsmodels.tsa.stattools import acf, pacf
import mpl_interactions.ipyplot as iplt
from sklearn.metrics import r2_score
from scipy.stats import linregress
from scipy.signal import detrend

ERROR: Error in parse(text = x, srcfile = src): <text>:1:8: unexpected symbol
1: import numpy
           ^


In [ ]:
def psi(m, b):
    return (1/10) * b**(-1/10) * m**(-0.9)

def cumulativeM(m, b):
    return b**(-0.1) * m**(0.1)

def inverseCumM(u, b):
    return b * u**(10)

In [ ]:
def pdfLogSpace(x, xb, gamma):
    return 2**(- gamma) * gamma * np.exp(gamma*(- x + xb + np.exp(xb) - 2*np.exp(x))+ np.log(1 + 2 * np.exp(x))) 


def cdfLogSpace(x, xb, gamma):
    return 1  - 2**(- gamma) * np.exp(gamma * (-2 * np.exp(x) + np.exp(xb) - x + xb))


def  drawCDF(unif, gamma, xb):
    #if np.sign(cdfLogSpace(-100, gamma = gamma, xb = xb) - unif) * np.sign(cdfLogSpace(50, gamma = gamma, xb = xb) - unif) == 1:
        #print(gamma, xb, unif, cdfLogSpace(-200, gamma = gamma, xb = xb) - unif, cdfLogSpace(50, gamma = gamma, xb = xb) - unif)
    return root_scalar(lambda t: cdfLogSpace(t, gamma = gamma, xb = xb) - unif, bracket=[-400, 50], method='brentq').root

In [ ]:
criticalPoint = 1 / np.log(2)
gammaValues = - np.abs(np.logspace(np.log10(np.abs(1 - criticalPoint)), np.log10(1e-3), 100))  + criticalPoint
gamma_gammaC = np.abs(np.logspace(np.log10(np.abs(1 - criticalPoint)), np.log10(1e-3), 100))

In [ ]:
def sizeAtBirth(gamma, xb0, seriesLength = 5000):
    uniformDraws = np.random.uniform(size = seriesLength)

    logSizes = np.zeros(seriesLength)
    logSizes[0] = xb0
    for i in range(1, seriesLength):
        logSizes[i] = drawCDF(unif = uniformDraws[i], gamma = gamma, xb = logSizes[i-1])

    return logSizes

In [ ]:
b = 10
seriesLength = 20400
simulate = False
startingPointsUniform = np.random.uniform(0, 1, len(gammaValues))
b = 10
def inverseCumM(u, b):
    return b * u**(10)
startingPoints = inverseCumM(startingPointsUniform, b)
if simulate:
    allSizes = np.zeros((len(gammaValues), seriesLength))
    for i, gamma in enumerate(tqdm_notebook(gammaValues)):
        sizeBirth = sizeAtBirth(gamma = gamma, xb0 = startingPointsUniform[i], seriesLength = seriesLength)
        allSizes[i,:] = sizeBirth

    #np.save("../../data/sizeAtBirthPowerSpectrum.npy", allSizes)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
allSizes = np.load("../../data/sizeAtBirthPowerSpectrum.npy")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (10, 5))
ax.hist(np.exp(allSizes[-1,:]), bins = 200, density = True)
plt.show()

In [ ]:
timeSeries = allSizes[0,:]
generations = np.arange(1, len(timeSeries) + 1) # The "time" variable

dt = 1 # The sampling interval
N = len(timeSeries) # The number of samples
T = N * dt # The total time of the measurement
timeSeriesFourier = np.fft.fft(timeSeries - timeSeries.mean()) # The Fourier transform of the signal removing the 0Hz component